# Gunicorn

> Gunicorn (Green Unicorn) is a Python WSGI HTTP Server for UNIX that serves your Django application. When using Gunicorn with a Django REST Framework (DRF) project, you can handle HTTP requests efficiently and reliably.

- skip_showdoc: true
- skip_exec: true

## Installation

```sh
pip install gunicorn
```

## Basic Usage

> To run your Django application using Gunicorn, navigate to your project's root directory and use the following command

```sh
gunicorn your_project_name.wsgi:application

```

> Replace your_project_name with the name of your Django project.

## Configuration Options

> Gunicorn provides various options to configure the server:

### Binding to a Specific Address and Port

```sh
gunicorn --bind 0.0.0.0:8000 your_project_name.wsgi:application
```

### Number of Workers

> The number of worker processes for handling requests. More workers can handle more requests concurrently but consume more memory.

```sh
gunicorn --workers 3 your_project_name.wsgi:application

```

### Timeout

> The maximum number of seconds a worker can spend handling a request before it is killed and restarted.

```sh
gunicorn --timeout 30 your_project_name.wsgi:application

```

### Logging

> You can specify log files for Gunicorn

```sh
gunicorn --access-logfile access.log --error-logfile error.log your_project_name.wsgi:application

```

## Deployment with Systemd

> For a production deployment, it's common to use systemd to manage Gunicorn. Here's an example of a systemd service file

```ini
[Unit]
Description=gunicorn daemon
After=network.target

[Service]
User=your_user
Group=www-data
WorkingDirectory=/path/to/your/project
ExecStart=/path/to/your/venv/bin/gunicorn --workers 3 --bind unix:/path/to/your/project/gunicorn.sock your_project_name.wsgi:application

[Install]
WantedBy=multi-user.target

```

- Replace your_user with your system username.
- Replace /path/to/your/project with the path to your Django project.
- Replace /path/to/your/venv with the path to your virtual environment.

### After creating the service file, enable and start the service

```bash
sudo systemctl enable gunicorn
sudo systemctl start gunicorn

```

## Serving Static and Media Files

> Gunicorn does not serve static or media files in production. Use a web server like Nginx to handle static files and reverse proxy requests to Gunicorn

> Example Nginx configuration

```nginx
server {
    listen 80;
    server_name your_domain_or_IP;

    location / {
        proxy_pass http://unix:/path/to/your/project/gunicorn.sock;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
        proxy_set_header X-Forwarded-Proto $scheme;
    }

    location /static/ {
        alias /path/to/your/project/static/;
    }

    location /media/ {
        alias /path/to/your/project/media/;
    }
}

```

## Security Considerations

- Keep Gunicorn and your Django project dependencies up to date.
- Use HTTPS for secure communication.
- Use a firewall to restrict access to the Gunicorn server, allowing only the web server (e.g., Nginx) to communicate with it.

## Performance Tuning

- Number of Workers: Start with (2 x $num_cores) + 1 workers. Monitor and adjust based on your application's performance and server's resource availability.
- Worker Class: Use different worker classes based on your application's nature, e.g., gevent for async applications.
- Keep-Alive: Adjust the keep-alive setting for persistent connections, especially in high-traffic scenarios.

## Monitoring and Logging

- Use monitoring tools like New Relic, Datadog, or Prometheus to keep an eye on the performance and health of your application.
- Log files should be monitored for errors and access patterns.


## Common Issues and Troubleshooting

- 502 Bad Gateway: Often due to Gunicorn not running or configuration issues with the reverse proxy server.
- Slow Performance: Could be due to insufficient worker processes or server resources. Profiling the application can help identify bottlenecks.
- Memory Leaks: Regularly monitor memory usage and ensure proper resource management in the application.